In [6]:
! pip install -U datasets
from datasets import load_dataset

In [ ]:
#Review_dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Cell_Phones_and_Accessories", trust_remote_code=True)
Meta_dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_Cell_Phones_and_Accessories", split="full",trust_remote_code=True)

In [ ]:
# Filter reviews where helpful_vote > 100
helpful_reviews = [review for review in Review_dataset["full"] if review.get("helpful_vote", 0) > 1000]

# Display the first few results (if any)
print(helpful_reviews[:2])  # Prints first 5 reviews with helpful_vote > 50

In [ ]:
from datetime import datetime

# Extract and convert relevant fields
extracted_data = []
for item in helpful_reviews:
    extracted_data.append({
        "star_rating": item.get("rating"),
        "product_title": item.get("title"),
        "review_body": item.get("text"),
        "review_date": datetime.utcfromtimestamp(item.get("timestamp") / 1000).strftime('%Y-%m-%d'),
        "verified_purchase": item.get("verified_purchase"),
        "helpful_vote": item.get("helpful_vote"),
        "product_id": item.get("parent_asin"),
        "user_id": item.get("user_id"),
    })

In [ ]:
! pip install textblob
from textblob import TextBlob
SA_Textblob=pd.DataFrame(extracted_data)
# Apply TextBlob polarity
SA_Textblob['sentiment_score'] = SA_Textblob['review_body'].apply(lambda x: TextBlob(x).sentiment.polarity)

# Classify sentiment
SA_Textblob['sentiment_label'] = SA_Textblob['sentiment_score'].apply(
    lambda x: 'positive' if x > 0.05 else 'negative' if x < -0.05 else 'neutral'
)

SA_Textblob

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

analyzer = SentimentIntensityAnalyzer()

# Apply sentiment analysis
SA_VADER = pd.DataFrame(extracted_data)
SA_VADER['sentiment_score'] = SA_VADER['review_body'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

# Classify sentiment
SA_VADER['sentiment_label'] = SA_VADER['sentiment_score'].apply(
    lambda x: 'positive' if x > 0.05 else 'negative' if x < -0.05 else 'neutral'
)

SA_VADER.head()

In [ ]:
SA_VADER

In [ ]:
# Merge all three on 'product id' and 'sentiment label'
merged = SA_VADER.merge(
    SA_Textblob[['product_id','user_id' ,'sentiment_label']],
    on=['product_id','user_id' ,'sentiment_label'])
merged=merged[((merged['star_rating'] > 3) & (merged['sentiment_label'] == 'positive')) |
       (merged['star_rating'] < 3) & (merged['sentiment_label'] == 'negative')]
merged.to_csv("df_review.csv", index=True)
merged

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import seaborn as sns

vectorizer = CountVectorizer(strip_accents=ascii, stop_words='english', token_pattern='(?u)\\b\\w\\w+\\b',ngram_range=(2,2), analyzer='word', max_df=0.7, min_df=1, max_features=None, vocabulary=None)

X = vectorizer.fit_transform(merged['review_body'])
ngrams = vectorizer.get_feature_names_out()

# Create frequency DataFrame
ngram_freq = pd.DataFrame(X.toarray(), columns=ngrams).sum().sort_values(ascending=False)
top_ngrams = ngram_freq.head(10)  # Get top 10 most frequent n-grams

# Visualization
plt.figure(figsize=(10, 6))
sns.barplot(x=top_ngrams.values, y=top_ngrams.index, palette='viridis')
plt.title('Top 10 Most Frequent Bi-grams in Product Features')
plt.xlabel('Frequency')
plt.ylabel('Bi-gram')
plt.tight_layout()
plt.show()

# Display results
print("Top 10 Bi-grams:")
print(top_ngrams)

In [ ]:
# n- gram analysis

import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

def generate_N_grams(text,ngram=2):
  text = text.lower()
  text = re.sub(r'<.*?>', '', text) # Remove HTML
  text = re.sub(r'[^a-z\s]', '', text) # Remove punctuation
  words=[word for word in text.split(" ") if word not in set(stopwords.words('english'))]
  #print("Sentence after removing stopwords:",words)
  temp=zip(*[words[i:] for i in range(0,ngram)])
  ans=[' '.join(ngram) for ngram in temp]
  return ans

In [ ]:
from collections import defaultdict

positiveValues= defaultdict(int)
#get the count of every word in both the columns of df_train and df_test dataframes where sentiment="positive"
for text in merged[merged.sentiment_label=="positive"].review_body:
  for word in generate_N_grams(text):
    positiveValues[word]+=1

df_positive=pd.DataFrame(sorted(positiveValues.items(),key=lambda x:x[1],reverse=True))

In [ ]:
negativeValues= defaultdict(int)

#get the count of every word in both the columns of df_train and df_test dataframes where sentiment="negative"
for text in merged[merged.sentiment_label=="negative"].review_body:
  for word in generate_N_grams(text):
    negativeValues[word]+=1

df_negative=pd.DataFrame(sorted(negativeValues.items(),key=lambda x:x[1],reverse=True))

In [ ]:
print(df_negative)
print(df_positive)